<a href="https://colab.research.google.com/github/ZhuRuoyu/computer-vision-waste-classification/blob/main/CV_FinalProject_WasteClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data description
https://www.kaggle.com/datasets/mostafaabla/garbage-classification

- 12 classes
- 15,150 images
- some examples: ...

# data prep

- data augmentation: create more samples for minority classes
- `split-folders` load train/test dataset


# modeling
try 3 transfer learning methods and compare the accuracies

  - CNN from scratch
  - VGG
  - ResNet
  - AlexNet




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
from torchsummary import summary #model summary

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install split-folders
import splitfolders
import os

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# data prep

- data augmentation (?)
- `splitfolders`
- `transform_scratch`, `transform_restnet`,
`transform_vgg`, `transform_alexnet`

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/ComputerVision-Course/final-project/

/content/drive/MyDrive/ComputerVision-Course


In [ ]:
# data augmentation
# ...

In [8]:
# folder name: garbage_classification

splitfolders.ratio('garbage_classification', output = 'output', seed = 2013,
                   ratio = (0.8, 0.2), group_prefix = None, move = False)

Copying files: 841 files [00:11, 72.26 files/s] 


In [ ]:
transform_scratch = transforms.Compose([transforms.ToTensor()])


transform_restnet = transforms.Compose([transforms.ToTensor(), # normalizes to range [0,1]
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) # further normalization
                       ])

transform_vgg = []

transform_alexnet = []

# functions prep
- function2trainTheModel
- visualization

In [ ]:
def function2trainTheModel(epochs = 50, model_name = 'vgg', learning_rate_alpha = 0.001, ):
  ''' a function that trains the model '''

  # number of epochs
  numepochs = epochs

  # create the model instance
  # ... change names
  if model_name == 'scratch':
    net = ConvolutionalNetwork()
  elif model_name == 'vgg':
    net = VGG16() # ... change name
  elif model_name == 'resnet':
    net = resnet()
  elif model_name == 'alexnet':
    net = alexnet()

  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate_alpha) # load hyperparameter



  # initialize losses
  losses    = torch.zeros(numepochs)
  trainAcc  = []
  testAcc   = []


  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    net.train() ## .train will backprop
    net.cuda()
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      X = X.cuda()
      y = y.cuda()

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      yHat = yHat.cpu()
      y = y.cpu()

      # compute accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    net.eval() ##.eval = not calculate gradient anymore
    X,y = next(iter(test_loader)) # extract X,y from test dataloader

     # New!
    X = X.cuda() #to(device)
    y = y.cuda() #to(device)

    with torch.no_grad(): # deactivates autograd
     yHat = net(X)

    # New! bring outputs back
    yHat = yHat.cpu()
    y = y.cpu()

    # compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )
    if (epochi % 10) == 0:

        print(f' epochs : {epochi}  loss : {loss : 2.2f}')
  # end epochs

  # function output
  return trainAcc,testAcc,losses,net


In [ ]:
# example - train model 1
trainAcc_resnet,testAcc_resnet,losses_resnet,net = function2trainTheModel(epochs = 50, model_name = 'resnet', learning_rate_alpha = 0.001, )

In [ ]:
def visualization(losses = losses_resnet, trainAcc = trainAcc_resnet,testAcc = testAcc_resnet, model_name = 'ResNet'):

  fig, ax = plt.subplots(1,2, figsize = (18,6)) # row = 1, col = 2

  # first plot: Training Loss v. Epochs
  ax[0].plot(losses, label = 'Train Loss')
  ax[0].set_xlabel('Epochs')
  ax[0].set_ylabel('Training Loss')
  ax[0].legend(fontsize = 15)
  ax[0].set_title('Training Loss v. Epochs')

  # second plot: Training & Testing Accuracy v. Epochs
  ax[1].plot(trainAcc, label = 'Train accuracy')
  ax[1].plot(testAcc, label = 'Test accuracy') #test_accuracy starts from 100?
  ax[1].set_xlabel('Epochs')
  ax[1].set_ylabel('Accuracy (%)')
  # print the latest accuracy [-1]
  ax[1].set_title(f'Train Accuracy: {trainAcc[-1]:.2f}% \n Test Accuracy: {testAcc[-1]:.2f}%')
  ax[1].legend(fontsize = 15)

  plt.show()

#Model 1: CNN from scratch


## 1. dataloader `transform = transforms.Compose([transforms.ToTensor()])`

## 2. modeling
- create model 1

## 3. train model 1
## 4. visualize the performance

## 1. load data into batches - model 1
batch size = 128

In [ ]:
#transform whole folder to tensor
#transform_scratch = transforms.Compose([transforms.ToTensor()])


# Prepare train test using ImageFolder
train_data = datasets.ImageFolder('output/train', transform = transform_scratch)
test_data = datasets.ImageFolder('output/val', transform = transform_scratch)


print(train_data.classes)
print(len(train_data))
print(test_data)

In [ ]:
# batch size = 128

torch.manual_seed(24)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=128, drop_last=True)

class_names = train_data.classes

print(class_names)

In [ ]:
#check train_loader (batch-size = 128, rgb=3, image= x )

for images, labels in train_loader:
  break
print(images.shape, '\n', labels)

In [ ]:
print('Label:', labels.numpy())
print('Class:', *np.array([class_names[i] for i in labels])) #labels in first batch

In [ ]:
len(labels)

In [ ]:
im = make_grid(images, nrow = 8) #images in first batch
#plt.imshow(np.transpose(im.numpy(), (1,2,0)))

In [ ]:
#3*227*227

## 2. create - model 1


In [ ]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # size = 28x28 -> 13x13 -> 5x5 (2 ConV and 2 pooling  is enough to reduce this image size)
        # self.conv1 = nn.Conv2d(1, 10, 5, 1,1)
        # 1 = channel, 10 = number of kernels, 5 = kernel_size（5x5 your choice), 1 = Stride, 1 = padding
        # size: np.floor( (28+2*1-5)/1 )+1 = 26/2 = 13 (/2 b/c maxpool)
        ## pooling is in the later code


        #self.conv2 = nn.Conv2d(10, 20, 5, 1,1)
        # 10 = channels or featuremaps ## match previous kernel numbers
        ## 20 = your choice, number of kernels
        # size: np.floor( (13+2*1-5)/1 )+1 = 11/2 = 5 (/2 b/c maxpool)

        # There are 20 kernels in the conv layer before the FC Layer
        # The dim of feature map after conv2 will be 5*5*20

        self.conv1 = nn.Conv2d(1, 10, 5, 1,1)
        self.conv2 = nn.Conv2d(10, 20, 5, 1,1)

        self.fc = nn.LazyLinear(64) #has warning if not running on colab
        self.output = nn.Linear(64,12)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)


        X = torch.flatten(X, 1) # Will be used with LazyLinear

        X = self.fc(X)
        X = F.relu(X)
        X = self.output(X)
        return X



In [ ]:
torch.manual_seed(101)
net = ConvolutionalNetwork()
net

## 3. train - model 1

In [ ]:
trainAcc,testAcc,losses,net = function2trainTheModel(epochs = 50, model_name = 'scratch', learning_rate_alpha = 0.001, )

## 4. visualization - model 1

### Training Loss v. Epochs
- x-axis: epoches
- y-axis: training loss:  losses[epochi] = np.mean(batchLoss)

### Training & Testing Accuracy v. Epochs

- x-axis: epoches
- y-axis: trainAcc.append / testAcc.append

In [ ]:
visualization(losses = , trainAcc = ,testAcc = , model_name = '...')

# Model 2: ResNet (transfer learning)

## 1. load data  with restnet transform

## 2. ResNet
- load ResNet: `resnet = torchvision.models.resnet18(pretrained=True)`
- freeze layers / unfreeze layers

## 3. train the model 2
## 4. visualize the performance

## 1. load data into batches - model 2

In [ ]:
# Transformations
# transform_restnet = transforms.Compose([ transforms.ToTensor(), # normalizes to range [0,1]
#                         transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) # further normalization
#                        ])


# import the data and simultaneously apply the transform
trainset = torchvision.datasets.STL10(root='./data', download=True, split='train', transform=transform_restnet)
testset  = torchvision.datasets.STL10(root='./data', download=True, split='test',  transform=transform_restnet)


# transform to dataloaders
batchsize    = 128
train_loader = DataLoader(trainset,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(testset, batch_size=batchsize,shuffle=True,drop_last=True)

In [ ]:
# check out the shape of the datasets
print('Data shapes (train/test):') # Here the data is not normalized. it will be normalized when it comes out of Loader
print( trainset.data.shape )
print( testset.data.shape )

# and the range of pixel intensity values
print('\nData value range:')
print( (np.min(trainset.data),np.max(trainset.data)) )

# the unique categories
print('\nData categories:')
print( trainset.classes )

## 2. load ResNet - model 2
- freeze/unfreeze layers

In [ ]:
resnet = torchvision.models.resnet18(pretrained=True)

## 3. train - model 2

In [ ]:
trainAcc,testAcc,losses,net = function2trainTheModel(epochs = 50, model_name = 'resnet', learning_rate_alpha = 0.001, )

## 4. visualization - model 2

In [ ]:
visualization(losses = losses_resnet, trainAcc = trainAcc_resnet,testAcc = testAcc_resnet, model_name = 'ResNet')

# Model 3: VGG (transfer learning)

# Model 4: AlexNet (transfer learning)